## **Task 1 - Flight Delay Prediction Model**

````
Flight delay prediction model is a trained machine learning model which predict that the flight you what to know will be delayed or not based 
on past data of flights . In this script we wil first process our data in accordance to our model and to get insights on how flights were 
delayed and various other factors and then train various machine learning classification model such as logistic regression, tree classifier,
 random forest classifier to gain best results.
````
Dataset used is [Airlines_delay](https://www.kaggle.com/datasets/ulrikthygepedersen/airlines-delay)



Some important libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# for pre processing data
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import resample
import scipy.stats as stats

# for statistical analysis
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,f1_score,precision_score,recall_score
from sklearn.model_selection import cross_val_score

# for various classification
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, VotingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier

In [2]:
# importing dataset
flight_data = pd.read_csv('./airlines_delay.csv')
flight_data.shape

(539382, 8)

Exploratory Data Analysis-(EDA)

In [ ]:
flight_data.head()

In [ ]:
#checking for any null values in our data
flight_data.isnull().sum()

In [ ]:
# Gaining various information for our data

In [ ]:
flight_data.columns

In [ ]:
flight_data.info()

In [ ]:
flight_data.describe()

Visulization of our data to get insights

In [ ]:
sns.countplot(x = 'Airline', hue = 'Class', data = flight_data)

In [ ]:
sns.barplot(x = 'Airline',y='Length', data = flight_data)

In [ ]:
sns.barplot(x = 'Airline', y = 'Time', data = flight_data)

*From above illuistration we get insights on data such as:
WN is highest delay than other airliness,
CO and UA have longest average flight time,
Almost all of the flight have same departure*

Preprocessing of our given data

In [3]:
#removing un required data in our model
flight_data = flight_data.drop('Flight', axis =1)
flight_data.head()

,Time,Length,Airline,AirportFrom,AirportTo,DayOfWeek,Class
0,1296.0,141.0,DL,ATL,HOU,1,0
1,360.0,146.0,OO,COS,ORD,4,0
2,1170.0,143.0,B6,BOS,CLT,3,0
3,1410.0,344.0,US,OGG,PHX,6,0
4,692.0,98.0,FL,BMI,ATL,4,0


In [4]:
print(flight_data.dtypes)

Time           float64
Length         float64
Airline         object
AirportFrom     object
AirportTo       object
DayOfWeek        int64
Class            int64
dtype: object


Encoding of various data to make all data in int form for our models

In [5]:
#taking unique values of Airline and encode them
airline = flight_data['Airline'].unique()
airline



array(['DL', 'OO', 'B6', 'US', 'FL', 'WN', 'CO', 'AA', 'YV', 'EV', 'XE',
       '9E', 'OH', 'UA', 'MQ', 'AS', 'F9', 'HA'], dtype=object)

In [8]:
encoder = LabelEncoder()

In [8]:


flight_data['Airline'] = encoder.fit_transform(flight_data['Airline'])
airline_encoded = flight_data['Airline'].unique()
airline_dict = {airline[i]:airline_encoded[i] for i in range(len(airline))} 

# making dictionary for User input encodes
airline_codes = open('air_code.txt','wt')
airline_codes.write(str(airline_dict))
airline_codes.close()
flight_data['Airline'].unique()




array([ 5, 12,  3, 14,  8, 15,  4,  1, 17,  6, 16,  0, 11, 13, 10,  2,  7,
        9])

In [9]:
# taking unique values of AirportFrom and encode them
air_from = flight_data['AirportFrom'].unique()
air_from

array(['ATL', 'COS', 'BOS', 'OGG', 'BMI', 'MSY', 'EWR', 'DFW', 'BWI',
       'CRW', 'LGB', 'BIS', 'CLT', 'IAH', 'LAX', 'JAX', 'SAV', 'CLE',
       'FLL', 'SAN', 'BHM', 'ROC', 'DTW', 'STT', 'AUS', 'DCA', 'PHX',
       'EYW', 'IND', 'JFK', 'ORD', 'PBI', 'SFO', 'MIA', 'DSM', 'SLC',
       'PHL', 'BZN', 'GRB', 'MBS', 'SBA', 'TYS', 'MSP', 'DEN', 'SAT',
       'BUF', 'RIC', 'SEA', 'PDX', 'LAS', 'IAD', 'HNL', 'BDL', 'MOT',
       'PSE', 'CPR', 'SNA', 'STL', 'CVG', 'PIT', 'HSV', 'SGF', 'RDU',
       'MEM', 'KOA', 'ELP', 'SJU', 'JAN', 'AEX', 'LGA', 'RSW', 'MDT',
       'GUC', 'MKE', 'CAE', 'GRR', 'FAR', 'LIT', 'OMA', 'BNA', 'EVV',
       'RDD', 'OKC', 'ITO', 'SJC', 'MCO', 'LBB', 'CSG', 'OAK', 'PHF',
       'ABQ', 'SMF', 'FAY', 'ABI', 'MSO', 'MFE', 'GEG', 'MSN', 'TPA',
       'DAY', 'RNO', 'PVD', 'ALB', 'CHO', 'ONT', 'LIH', 'PSP', 'LAN',
       'LEX', 'XNA', 'GJT', 'CMH', 'GSO', 'PSC', 'SYR', 'AVL', 'MHT',
       'GRK', 'MCI', 'TXK', 'LRD', 'ABE', 'LWB', 'ERI', 'DAL', 'ANC',
       'TUS', 'ROA',

In [10]:
flight_data['AirportFrom'] = encoder.fit_transform(flight_data['AirportFrom'])
air_from_encoded = flight_data['AirportFrom'].unique()
air_from_dict = {air_from[i]:air_from_encoded[i] for i in range(len(air_from))} 
# for user input encodes
airfrom_codes = open('airfrom_code.txt','wt')
airfrom_codes.write(str(air_from_dict))
airfrom_codes.close() 

flight_data['AirportFrom'].unique()

array([ 16,  65,  35, 203,  32, 198,  96,  80,  45,  69, 160,  29,  60,
       135, 154, 147, 245,  58, 104, 243,  27, 238,  85, 269,  18,  78,
       217,  97, 139, 148, 208, 213, 253, 183,  84, 261, 216,  46, 117,
       171, 246, 285, 197,  79, 244,  43, 234, 252, 214, 153, 134, 128,
        22, 192, 225,  67, 264, 268,  71, 221, 132, 254, 233, 177, 150,
        91, 260, 146,  10, 159, 241, 174, 124, 184,  47, 119,  99, 162,
       205,  33,  94, 231, 204, 143, 258, 173, 155,  70, 202, 215,   2,
       262, 101,   1, 196, 178, 111, 195, 277,  76, 236, 228,  12,  53,
       207, 161, 227, 152, 157, 290, 114,  61, 120, 224, 273,  19, 182,
       118, 172, 283, 165,   0, 167,  92,  75,  14, 280, 237, 191, 149,
       248,  50, 279, 107, 106,  42, 102, 115, 230, 175, 109,  44, 223,
       240, 129,  34,  68,  38,  17, 256, 263, 232, 209, 116, 151, 136,
       242,  48, 137, 193, 288,  54, 170, 130,  20,  21, 278, 121, 126,
       188,  93,  13, 181, 141,  56, 190,  26, 267, 187,  89, 15

In [6]:
# taking unique values of Airport and encode them
air_to = flight_data['AirportTo'].unique()
air_to

array(['HOU', 'ORD', 'CLT', 'PHX', 'ATL', 'BHM', 'DFW', 'MEM', 'GRR',
       'PBI', 'MCO', 'SFO', 'DEN', 'YUM', 'BWI', 'HPN', 'EWR', 'JFK',
       'MKE', 'OAK', 'IAH', 'CLE', 'SYR', 'SJU', 'BDL', 'SAN', 'DTW',
       'PSP', 'DCA', 'LGA', 'STL', 'FAY', 'MSP', 'BUF', 'LAS', 'SGU',
       'SLC', 'GJT', 'LAX', 'VPS', 'FAR', 'RKS', 'BOS', 'ANC', 'SNA',
       'ONT', 'RNO', 'JAX', 'GSP', 'CVG', 'TPA', 'SEA', 'LEX', 'SMF',
       'CAE', 'STT', 'DAY', 'MDW', 'RSW', 'ITO', 'IAD', 'ICT', 'HNL',
       'MIA', 'CRW', 'RDU', 'MHT', 'FAT', 'CAK', 'COS', 'DAL', 'TYS',
       'PHL', 'ABI', 'MOB', 'SDF', 'SAV', 'MDT', 'LIT', 'TUL', 'ACV',
       'BNA', 'MCI', 'MSY', 'FLL', 'PVD', 'OKC', 'ECP', 'PHF', 'AUS',
       'RIC', 'LIH', 'ABQ', 'JAN', 'PIT', 'BMI', 'BTV', 'RAP', 'MRY',
       'CSG', 'SHV', 'FAI', 'SJC', 'PIA', 'SBN', 'IND', 'SGF', 'ACT',
       'SRQ', 'ROC', 'CHO', 'JAC', 'SAT', 'FWA', 'OMA', 'PDX', 'CMH',
       'PWM', 'CID', 'TRI', 'ORF', 'GTF', 'TUS', 'MHK', 'BUR', 'MLU',
       'CEC', 'TEX',

In [9]:
flight_data['AirportTo'] = encoder.fit_transform(flight_data['AirportTo'])
air_to_encoded = flight_data['AirportTo'].unique()
air_to_dict = {air_to[i]:air_to_encoded[i] for i in range(len(air_to))}

airto_codes = open('airto_code.txt','wt')
airto_codes.write(str(air_to_dict))
airto_codes.close()
flight_data['AirportTo'].unique()

array([129, 208,  60, 217,  16,  27,  80, 177, 119, 213, 173, 253,  79,
       292,  45, 130,  96, 148, 184, 202, 135,  58, 273, 260,  22, 243,
        85, 227,  78, 159, 268, 101, 197,  43, 153, 255, 261, 114, 154,
       288,  99, 235,  35,  14, 264, 207, 236, 147, 121,  71, 277, 252,
       157, 262,  47, 269,  76, 175, 241, 143, 134, 136, 128, 183,  69,
       233, 182, 100,  48,  65,  75, 285, 216,   1, 190, 251, 245, 174,
       162, 279,   6,  33, 172, 198, 104, 228, 204,  87, 215,  18, 234,
       161,   2, 146, 221,  32,  42, 230, 194,  70, 256,  98, 258, 218,
       247, 139, 254,   5, 267, 238,  53, 145, 244, 109, 205, 214,  61,
       229,  56, 278, 209, 122, 280, 181,  44, 188,  51, 274, 171,  84,
       131, 158,  91,  10,  67, 155, 200,  12,  66, 166,  52, 187, 111,
        21, 179,  41, 103, 151, 224, 120, 203, 195, 116, 223, 232,  46,
        82,  68, 283, 150, 193, 170, 275, 290,  72, 248,  24,  83, 289,
        81, 266,  13,  88,  23, 102,  93,  94,  19, 132, 219, 18

In [ ]:
flight_data.dtypes

### ReSampling Our Train Data

In [ ]:
sns.countplot(x='Class',data = flight_data)
flight_data['Class'].value_counts()

In [ ]:
# Dividing data into majority and minority
flight_major = flight_data[(flight_data['Class']==0)]
flight_minor = flight_data[(flight_data['Class']==1)]
# Umsampling minor data

flight_minor_upsample = resample(flight_minor,replace = True ,n_samples = len(flight_major),random_state = 0)

# combining both data

flight_upsampled = pd.concat([flight_minor_upsample, flight_major])

In [ ]:
sns.countplot(x='Class',data = flight_upsampled)
flight_upsampled['Class'].value_counts()

Removing Outliers from our data

In [ ]:
z= np.abs(stats.zscore(flight_upsampled))
final_train = flight_upsampled[(z<3).all(axis=1)]
final_train.shape

Attribute Correlation

In [ ]:
sns.heatmap(final_train.corr(), fmt = '.2g')

Splitting data into train and test data

In [ ]:
#dividing data into dependent and undependent feature 
x = final_train.drop('Class', axis=1)
y = final_train['Class']

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(x,y, test_size=0.2, random_state=42)

### Implementation of various models



***Logistic Regression***

In [ ]:
logistic_model = LogisticRegression()

In [ ]:
logistic_model.fit(x_train,y_train)

In [ ]:
log_pred =logistic_model.predict(x_test)
print("Accuracy Score : ",round(accuracy_score(y_test,log_pred)*100,2),'%')

In [ ]:
print("F-1 Score : ",round(f1_score(y_test,log_pred)*100,2),'%')
print("Precision Score : ",round(precision_score(y_test,log_pred)*100,2),'%')
print("Recall Score : ",round(recall_score(y_test,log_pred)*100,2),'%')

***DECISION TREE CLASSIFIER***

In [ ]:
dt_classifier= DecisionTreeClassifier()


dt_classifier.fit(x_train,y_train)


y_pred=dt_classifier.predict(x_test)
print("Accuracy Score : ", round(accuracy_score(y_test,y_pred)*100,2),'%')

In [ ]:
print("F-1 Score : ",round(f1_score(y_test,y_pred)*100,2),'%')
print("Precision Score : ",round(precision_score(y_test,y_pred)*100,2),'%')
print("Recall Score : ",round(recall_score(y_test,y_pred)*100,2),'%')


***Random Forest Classifier***

In [ ]:
rf_classifier=RandomForestClassifier(random_state=0)


rf_classifier.fit(x_train,y_train)


rf_predict=rf_classifier.predict(x_test)
print("Accuracy Score : ", round(accuracy_score(y_test,rf_predict)*100,2),'%')

In [ ]:
print("F-1 Score : ",round(f1_score(y_test,rf_predict)*100,2),'%')
print("Precision Score : ",round(precision_score(y_test,rf_predict)*100,2),'%')
print("Recall Score : ",round(recall_score(y_test,rf_predict)*100,2),'%')

***AdaBoost Classifier***

In [ ]:
ada_model= AdaBoostClassifier()
ada_model.fit(x_train,y_train)
ada_pred=ada_model.predict(x_test)
print("Accuracy Score : ", round(accuracy_score(y_test,ada_pred)*100,2),'%')

In [ ]:
print("F-1 Score : ",round(f1_score(y_test,ada_pred)*100,2),'%')
print("Precision Score : ",round(precision_score(y_test,ada_pred)*100,2),'%')
print("Recall Score : ",round(recall_score(y_test,ada_pred)*100,2),'%')

***Naive Bayes***

In [ ]:
gnb = GaussianNB()
gnb.fit(x_train, y_train)
y_pred = gnb.predict(x_test)
print("Accuracy Score : ", round(accuracy_score(y_test,y_pred)*100,2),'%')

In [ ]:
print("F-1 Score : ",round(f1_score(y_test,y_pred)*100,2),'%')
print("Precision Score : ",round(precision_score(y_test,y_pred)*100,2),'%')
print("Recall Score : ",round(recall_score(y_test,y_pred)*100,2),'%')

***KNN Classifer***

In [ ]:
knn = KNeighborsClassifier()
knn.fit(x_train, y_train)
y_pred = knn.predict(x_test)
print("Accuracy Score : ", round(accuracy_score(y_test,y_pred)*100,2),'%')

In [ ]:
print("F-1 Score : ",round(f1_score(y_test,y_pred)*100,2),'%')
print("Precision Score : ",round(precision_score(y_test,y_pred)*100,2),'%')
print("Recall Score : ",round(recall_score(y_test,y_pred)*100,2),'%')

***Voting Classifier***

In [ ]:
model_1 = LogisticRegression()

model_1.fit(x_train, y_train)

ypred = model_1.predict(x_test)


model_2 = GaussianNB()

model_2.fit(x_train, y_train)

ypred = model_2.predict(x_test)


model_3 = RandomForestClassifier()

model_3.fit(x_train, y_train)

ypred = model_3.predict(x_test)


eclf = VotingClassifier(

 estimators=[('lr', model_1), ('rf', model_2), ('gnb', model_3)],

 voting='hard'

)



In [ ]:
for clf, label in zip([model_1, model_2, model_3, eclf], ['Logistic Regression', 'Naive Bayes', 'Random Forest', 'Ensemble']):

 scores = cross_val_score(clf, x, y, scoring='accuracy', cv=5)

 print("Accuracy: %0.2f (+/- %0.2f) [%s]" % (scores.mean(), scores.std(), label))

| Model | Testing Accuracy | F-1 Score|Precision Score|Recall Score|
|----------|---------|-------------------------|-|-|
| Logistic Regression	| 57.1% | 57.3% |57.55%|57.1%|
|DECISION TREE CLASSIFIER 	| 68.23%	| 67.61% | 68.67% |  68.58% |
|Random Forest Classifier| 70.07%	| 70.89% | 68.74%  | 73.18% |
|AdaBoost Classifier 	| 62.79%	| 61.54% | 63.42%|59.76%|
| Naive Bayes| 57.99%	| 58.97% | 57.42%|60.41%|
|KNN Classifer 	|  67.66%	| 68.44% | 66.6%|70.38%|



*Via performing on various model we get that Random Forest Classifer performs best in our case *

### Visualizing Our Training Model

In [ ]:
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_test,rf_predict)
plt.figure(figsize=(7,7))
sns.heatmap(data=cm,linewidth=3, annot=True, square=True, cmap='Blues')
plt.xlabel("Predicted label")
plt.ylabel("Actual label")
all_sample_title = 'Accuracy Score : {0}'.format(rf_classifier.score(x_test,y_test))
plt.title(all_sample_title, size=15)

In [ ]:
import joblib
joblib.dump(rf_classifier,"Flight.pkl")  ## Saving our model for future use


## Building GUI For RandomForest Classifier

In [1]:
import gradio as gr

import datetime
import time
import joblib

week = {"Monday":1,"Tuesday":2,"Wednesday":3,'Thursday':4,'Friday':5,"Saturday":6,'Sunday':7}

def prediction(t,len,air,airfrom,airto,day):
    x = time.strptime(t,'%H:%M:%S')
    t = datetime.timedelta(hours=x.tm_hour,minutes=x.tm_min,seconds=x.tm_sec).total_seconds()/60    
    dataf = {
        'Time' : t,
        'Length' : len,
        'Airline' : [airline_dict[air]],
        'AirportFrom': [air_from_dict[airfrom]],
        'AirportTo'  : [air_to_dict[airto]],
        'DayOfWeek': week[day]
    }
    d_f = pd.DataFrame(dataf)

    model = joblib.load('Flight.pkl')
    if model.predict(d_f) == 1:
        return "Your Flight will be probably delayed"
    return "Your Flight will be on time"




In [ ]:
# pip freeze

In [2]:

# gr.Base.from_hub("dawood/microsoft_windows ").

face = gr.Interface(
    prediction,
    [gr.Textbox(
    label= 'Time of the Flight',
    info = 'Should be in HH:MM:SS in 24hrs format',
    lines = 1,
    value='17:15:00',
    ),
    gr.Number(value=100,label="Length of Flight"),
    gr.Textbox(
    label="Code of Airline",
    lines=1,
    value="OO",
    ),
    gr.Textbox(
    label=" Departure Airport",
    info='Airport Code',
    lines=1,
    value='MSP',
    ),
    gr.Textbox(
    label='Arrival Airport',
    info='Airport Code',
    lines=1,
    value='MOT',
    ),
    gr.Dropdown(['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday'],value="Monday",label="Day")




    ],title="Flight Delay Prediction", theme = gr.themes.Base.from_hub("freddyaboulton/test-blue") ,outputs="text"
    
)
face.launch(inbrowser=True)

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


Traceback (most recent call last):
  File "c:\Users\anees\AppData\Local\Programs\Python\Python310\lib\site-packages\gradio\routes.py", line 401, in run_predict
    output = await app.get_blocks().process_api(
  File "c:\Users\anees\AppData\Local\Programs\Python\Python310\lib\site-packages\gradio\blocks.py", line 1302, in process_api
    result = await self.call_function(
  File "c:\Users\anees\AppData\Local\Programs\Python\Python310\lib\site-packages\gradio\blocks.py", line 1025, in call_function
    prediction = await anyio.to_thread.run_sync(
  File "c:\Users\anees\AppData\Local\Programs\Python\Python310\lib\site-packages\anyio\to_thread.py", line 31, in run_sync
    return await get_asynclib().run_sync_in_worker_thread(
  File "c:\Users\anees\AppData\Local\Programs\Python\Python310\lib\site-packages\anyio\_backends\_asyncio.py", line 937, in run_sync_in_worker_thread
    return await future
  File "c:\Users\anees\AppData\Local\Programs\Python\Python310\lib\site-packages\anyio\_backe